# Introduction to management optimization (under uncertainty)

So far in these awesome tutorial notebooks, we have been focused on estimating uncertain model inputs (i.e. parameters), estimating the uncertainty in these inputs, and propagating the unceratinty in these inputs to important model outputs (i.e. predictions), namely unmeasured groundwater levels and surface-water/groundwater exchange fluxes during (unseen) dry conditions.  Now we are going to go the next level and start to use the model in an entirely new way:  rather than estimating the uncertainty in important model outputs during unseen dry conditions, what if we "flip the script" and instead ask the question "how much groundwater can we extract before the surface-water/groundwater exchange flux reaches an unacceptable value"?  And what if we augment the question with something even more exciting: "...at the 95% confidence level"?  OOOhhh that sounds so nice!  

Let's begin our journey with some terminology:

    - "parameter": an uncertain model input whose value we want to estimate and whose uncertainty we want to propagate to important model outputs
    - "decision variable": a model input whose value can be "controlled" by human activity.  For example, groundwater extraction rates or surface-water structure opertions.  Like a parameter, a decision variable also influences important model outputs.
    - "constraint": an uncertain model output whose real-world equivalent value has a range of "undesired values".  In management optimization, "constraints" are typically "inequality" constraints, meaning the constraint can take any value other than the undesired values.  Think "surface-water/groundwater exchange flux must be greater than XXX to support ecological flows".
    - "chance constraint": given that the relation between decision variables and constraints must be evaluated with the model, constraints are "uncertain" in exactly the same way the "predictions" or "forecasts" are uncertain. Therefore, it only makes sense that we include "uncertainty" in the management optimization process.  One way to do this is with "chance constraints", where we include uncertainty in the constraints.  However, most management optimization algorithms do not tolerate a statistical distribution of constraint values - they need a single value.  So we will use the concept of "risk" to identify a scalar constraint value from the statistical constraint distribution - a scalar value that implicitly represents the underlying uncertainty.
    - "risk": a value that ranges for 0.0 (risk tolerant) to 1.0 (risk averse). When you see "risk" think "reliabiliy". The more risk averse you are, the more "uncertainty" will cost you in the final optimization solution.
    - "objective function": a (potentially nonlinear) function of the decision variables that is to be maximized or minimized, depending on the problem.  For example, in the case of groundwater extraction, the objective is to maximize the volume of groundwater extracted (subject to not violating the constraints).
    

Whew!  Ok, lets see some of this in practice...


### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see "intro to model" notebook), and the high-dimensional PEST dataset prepared in the "pstfrom pest setup" and "obs and weights" notebooks. 

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
sys.path.insert(0,os.path.join("..", "..", "dependencies"))
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd



To maintain continuity in the series of tutorials, we we use the PEST-dataset prepared in the "obs and weigths" tutorial. Run the next cell to copy fthe necessary files across. Note that if you will need to run the previous notebooks in the correct order beforehand.

Specify the path to the PEST dataset template folder. Recall that we will prepare our PEST dataset files in this folder, keeping them separate from the original model files. Then copy across pre-prepared model and PEST files:

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_template')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("..","part2_2_obs_and_weights","freyberg6_template")
if not os.path.exists(org_t_d):
    raise Exception("you need to run the '/part2_2_obs_and_weights/freyberg_obs_and_weights.ipynb' notebook")

shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'freyberg_mf6.pst')

### Inspect the PEST Dataset

OK. We can now get started.

Load the PEST control file as a `Pst` object. We are going to use the PEST control file that was created in the "pstfrom pest setup" tutorial. This control file has observations with weights equal to the inverse of measurement noise (**not** weighted for visibility!).

In [ ]:
pst = pyemu.Pst(pst_path)

In [ ]:
# check to see if obs&weights notebook has been run
if not pst.observation_data.observed.sum()>0:
    raise Exception("You need to run the '/part2_2_obs_and_weights/freyberg_obs_and_weights.ipynb' notebook")

Make a quick parameter summary table as a reminder of what we have in our control file:

In [ ]:
pst.write_par_summary_table(filename="none")

Recall that our parameterisation is quite comprehensive, with pilot points and grid based (e.g. cell-by-cell) parameters. 

Let's recall how many adjustable parameters we have:

In [ ]:
pst.npar_adj

### Moving from DA/UQ to OPT: decision variables

As previously discussed, we are now moving away from the data assimilation and uncertainty analysis focus to one of management optimization.  Fortunately for you, we have been planning this journey all along and have included all the nessecary pieces for management optimization with in the PEST interface we have been using - wat?!  How can this be?! Well, for starters, we have been estimating uncertain historical and future groundwater extraction rates (the `WEL` package "flux" quantities). While it is obvs important to account for this important source of model input uncertainty, these same "parameters" can also be recast as "decision varibles" - they are one in the same, depending on the analysis you are interested in...

Let's examine these important model inputs:

In [ ]:
par = pst.parameter_data
wpar = par.loc[par.parnme.str.contains("welgrd"),:]
wpar

We see that the "grid-scale" well flux parameters are just the thing we are wanting to use as decision variables.  But - and this is important - we want to only optimize future water use subject to future surface-water/groundwater exchange constraints.  This means we want to only treat "future" water use as decision variables, where "future" refers to stress periods 13 thru 25 (remember all that from the previous notebooks?).  So we want to fix all "parameters" except the grid-scale well flux parameters for stress periods 13 thru 25.  The `inst` metadata gives us a zero-based stress period tag (lucky us!):

In [ ]:
wpar.loc[:,"inst"] = wpar.inst.astype(int)
par.loc[:,"partrans"] = "fixed"
future_wpar_names = wpar.loc[wpar.inst >= 13,"parnme"]
future_wpar_names.values

Now we are going to do some trickery to help us use these "parameters" as decision variables, mostly related to the bounds and transform; we want to be able to "turn off" a given extraction well, so we want its transform to be `none` and we want it's lower bound to 0.0 (remember these are actually multiplier parameters).

In [ ]:
par.loc[future_wpar_names,"partrans"] = "none"
par.loc[future_wpar_names,"parlbnd"] = 0.0
par.loc[future_wpar_names,"parval1"] = 0.0
par.loc[future_wpar_names,"parubnd"] = 6.0
par.loc[future_wpar_names,"pargp"] = "decvars"

Now we need to set just one little "++" arg:

In [ ]:
pst.pestpp_options["opt_dec_var_groups"] = "decvars"

### Moving from DA/UQ to OPT: decision variables: objective function

Its an unfortunately naming issue but the "objective function" we have talked about for many, many notebooks will now mean something completely different.  As a quick aside, its important to realize that the reason we talk about the "objective function" in the previous notebooks and thru out the PEST world is because all of the PEST and PEST++ tools for parameter estimation/data assimilation/uncertainty quantification are in fact optimization algorithms that have been repurposed for these analyses.  But, unlike before, where we had to worry about overfitting, parameter plausibility, etc, we are now free to adjust decision variables in any pattern we like and we are free to seek the absolute best objective function value.  

So what should our objective function be?!  Well, our management objective is to maximize extracted groundwater.  So the objective function is a simple sum of the decision variable values - easy as!  So how does this look in the PEST interface?  Answer:  prior information equation:

In [ ]:
pst.add_pi_equation(future_wpar_names,pilbl="obj_well",obs_group="greater_than")

In [ ]:
pst.prior_information.equation[0]

yikes!  that looks like a bunch of junk but really its a simple equation summing all of the future water use decision variable values.  The right-hand side value of 0.0 is unused; we tagged this prior info equation with a "greater_than" tag for two reasons:  0) to remind us what we want maximize this value and 1) for later, more advanced optimization analyses.  For now, to use the most basic optimization tool in pest++, we will pass the objective direction directly to the algorithm:

In [ ]:
pst.pestpp_options["opt_direction"] = "max"
pst.pestpp_options["opt_objective_function"] = "obj_well"

### Moving from DA/UQ to OPT: decision variables: objective function

This is where things get interesting.  If we were to run the optimization now, the algorithm would simply take each decision variable to its upper bound since that is the maximum of the objective function.  But this would obvs not be acceptable because of ecological impacts (the surface-water/grounwater exchange would be not-good!)  So we need to "constrain" the optimization problem.  

A bit more terminology: a "feasible" solution is one where the decision variables yield a simulation result that satisfies all of the constraints.  

Let's now setup "model-based" or observation constraints.  We are going to target the "incremental" global water budget SFR components - these represent the flux of groundwater to surface-water.  Note the sign convention is negative means from groundwater to surface-water.  So to maintain ecological flows, we want to keep a (substantially) negative value.  

In [ ]:
obs = pst.observation_data
swgw_obs = obs.loc[obs.obsnme.str.contains("inc") & obs.obsnme.str.contains("sfr"),:]
swgw_obs

In [ ]:
swgw_obs.obsval.plot(figsize=(15,3))

So we see that in stress periods 13 thru 25, the incremental surface flux gets pretty close to 0.0, the point where groundwater stops contributing to surface-water.  So lets form an interesting optimization problem:  the incremental SFR flux must be "less than" -100 $\frac{L^3}{T}$.  We tell the PEST++ optmization tools about these constraints by tagging their observation group name with "less_than" and giving them non-zero weight:

In [ ]:
swgw_obs.loc[:,"totim"] = swgw_obs.totim.astype(float)
swgw_constraint_names = swgw_obs.loc[swgw_obs.totim > 4000,"obsnme"]

In [ ]:
obs.loc[:,"weight"] = 0.0 #deactivate all existing non-zero weighted observations
obs.loc[swgw_constraint_names,"weight"] = 1.0 # just non-zero, the value doesnt matter...
obs.loc[swgw_constraint_names,"obsval"] = -250.0 # the constraint right hand side

obs.loc[swgw_constraint_names,"obgnme"] = "less_than_swgw"



Great! but there is another problem:  what if there is a minimum flux of groundwater that is needed?  So for municipal use?  This optimization problem will just turn groundwater extraction rates way down during the dry season to meet ecological constraints, but that might not be enough drinking water - the classic competition for resources problem.  Let's add some more constraints to make sure a minimum amount of groundwater is also produced.  We can do this using the incremental WEL budget component or we can add additional prior information equations, one for each future stress period.  Its probably easiest to just use the incremental budget obs:

In [ ]:
wel_obs = obs.loc[obs.obsnme.str.contains("inc") & obs.obsnme.str.contains("wel"),:]
wel_obs

We see an observed value of -2350.0 during the future stress periods so let's go with half of that, assuming that number is meaningful (?).  

In [ ]:
wel_obs.loc[:,"totim"] = wel_obs.totim.astype(float)
wel_constraint_names = wel_obs.loc[wel_obs.totim>4000,"obsnme"]

In [ ]:
obs.loc[wel_constraint_names,"weight"] = 1.0
obs.loc[wel_constraint_names,"obgnme"] = "less_than_wel" #again, negative means out of groundwater
obs.loc[wel_constraint_names,"obsval"] = -2350.0 #again, negative means out of groundwater


### Run PESTPP-OPT

`PESTPP-OPT` implements a sequential linear programming algorithm.  The linear programming solve is done using the simplex algorithm (not the Nelder-Meade simplex, the Danzig simplex).  This algorithm requires a "response" matrix - a matrix the maps the linear relation between decision variables and constraints.  Sound familiar?  Its exactly the same concept as the jacobian matrix and we will again rely on finite difference derivatives to fill this matrix - one model run per decision variable.  But, unlike a jacobian used for parameter estimation/data assimilation, we want to make sure the decision variable pertubation is large enough that we have a representative response at the constraints. So to do this, we will use some of the derivative calculation controls in the `* parameter groups` section:





In [ ]:
pst.rectify_pgroups()
pst.parameter_groups

In [ ]:
pst.parameter_groups.loc["decvars","inctyp"] = "absolute"
pst.parameter_groups.loc["decvars","derinc"] = 1.0 #remember these are multipliers!
pst.parameter_groups.loc["decvars","derinclb"] = 1.0

In [ ]:
pst.control_data.noptmax = 0
pst.write(pst_path,version=2)

In [ ]:
pyemu.os_utils.run("pestpp-opt freyberg_mf6.pst",cwd=t_d)

In [ ]:
pst.control_data.noptmax = 1
pst.write(pst_path,version=2)

# Attention!

You must specify the number which is adequate for ***your*** machine! Make sure to assign an appropriate value for the following `num_workers` variable:

In [ ]:
num_workers = 15 #psutil.cpu_count(logical=False) # update according to your available resources!

Then specify the folder in which the PEST manager will run and record outcomes. It should be different from the `t_d` folder. 

In [ ]:
m_d = os.path.join('master_opt_1')

The following cell deploys the PEST agents and manager and then starts the run using `pestpp-opt`. Run it by pressing `shift+enter`.

If you wish to see the outputs in real-time, switch over to the terminal window (the one which you used to launch the `jupyter notebook`). There you should see `pestpp-opt`'s progress. 

If you open the tutorial folder, you should also see a bunch of new folders there named `worker_0`, `worker_1`, etc. These are the agent folders. `pyemu` will remove them when PEST finishes running.

This run should take a while to complete (depending on the number of workers and the speed of your machine). If you get an error, make sure that your firewall or antivirus software is not blocking `pestpp-opt` from communicating with the agents (this is a common problem!).

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-opt","freyberg_mf6.pst",num_workers=num_workers,worker_root=".",
                           master_dir=m_d)

### Processing PESTPP-OPT

Ok, so now what? Well let's check out the constraints (since the include both the water use and sw-gw exchange fluxes).  Here are the files that might have what we need:

In [ ]:
[f for f in os.listdir(m_d) if f.endswith(".rei")]

Wat?! Whats with this "est" and "sim" stuff?  Well, in PESTPP-OPT, the linear-programming solution yields what it thinks the final constraint values should be, based on the assumed linearity of the response matrix - these are the "est"imated constraint values.  But we know that the relation between decision variables and constraints might be non-linear (nah, really?!).  So PESTPP-OPT actually "sim"ulates the model one last time with the optimal decision variable values to verify the results. (the ".jcb.rei" files are the simulation results where the response matrix was calculated).  Lets compare these:

In [ ]:
sim_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_mf6.1.est.rei"))
est_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_mf6.1.sim.rei"))
constraints = swgw_constraint_names.tolist()
constraints.extend(wel_constraint_names)
fig,ax = plt.subplots(1,1,figsize=(10,3))
sim_df.loc[swgw_constraint_names,"est"] = est_df.loc[swgw_constraint_names,"modelled"]
sim_df.loc[swgw_constraint_names,["modelled","est"]].plot(ax=ax,kind="bar")
ax.plot(ax.get_xlim(),[-250,-250],"k--")

Ok, so we see that there is some mild nonlinearity but we are still pretty close.  #winning

Hackery alert:  now lets visualize the pattern of groundwater use across the future stress periods and plot that with the constraint information:

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_mf6.par"))
par_df.loc[future_wpar_names,:]

In [ ]:
wpar = wpar.loc[future_wpar_names,:].copy()
wpar.loc[:,"kij"] = wpar.apply(lambda x: (x.idx0,x.idx1,x.idx2),axis=1)
wpar.loc[:,"optimal"] = par_df.loc[wpar.parnme,"parval1"]
wpar

In [ ]:
inst_vals = wpar.inst.unique()
inst_vals.sort()
inst_vals

In [ ]:
fig,axes = plt.subplots(1,len(inst_vals),figsize=(15,3))
colors = ["r","g","b","c","m","y","0.5"]
for inst,ax in zip(inst_vals,axes):
    ipar = wpar.loc[wpar.inst==inst,:].copy()
    ipar.sort_values(by="kij",inplace=True)
    ipar.index = ipar.kij
    ipar.optimal.plot(ax=ax,kind="bar",color=colors)
    ax.set_title("stress period {0}".format(inst+1),loc="left")
    ax.set_ylabel("extraction factor")
plt.tight_layout()

In [ ]:
cobs = obs.loc[constraints,:].copy()
cobs.loc[:,"totim"] = cobs.totim.astype(float)
totims = cobs.totim.unique()
totims.sort()
#print(cobs.totim)
fig, axes = plt.subplots(2,1,figsize=(15,3))
axes[0].plot(cobs.loc[swgw_constraint_names,"totim"].values,sim_df.loc[swgw_constraint_names,"modelled"].values)
axes[0].plot(cobs.loc[swgw_constraint_names,"totim"].values,cobs.loc[swgw_constraint_names,"obsval"].values)

axes[0].set_ylim(-500,0)
axes[0].plot(axes[0].get_xlim(),[-250,-250],"k--")
axes[1].plot(cobs.loc[wel_constraint_names,"totim"].values,sim_df.loc[wel_constraint_names,"modelled"].values)
axes[1].plot(axes[1].get_xlim(),[-2350,-2350],"k--")
axes[1].set_ylim(-8000,0)
axes[0].set_xticks(totims)
axes[1].set_xticks(totims)
axes[0].set_xlim(totims[0],totims[-1])
axes[1].set_xlim(totims[0],totims[-1])
axes[0].set_title("sw-gw constraints",loc="left")
axes[1].set_title("wel constraints",loc="left")

plt.tight_layout()